In [78]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnull, count, isnan, when, col, date_format, to_date

In [80]:
spark = SparkSession.builder.appName('myson').getOrCreate()

## READING AND UNDERSTANDING THE DATASET IN JSON FORMAT

In [81]:
trial_df=spark.read.json('dataset.json', multiLine= True)

In [82]:
trial_df.show(2)

+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|ageofparticipant|           clinician|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|              result|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|              19|{Ontario, Saul, t...|  Placebo|    1619827200000|      1617235200000|                            52|{BP normalized, r...|
|              14|{Ontario, Saul, n...| Naproxen|    1619827200000|      1617235200000|                            78|    {Follow-up, N/A}|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
only showing top 2 rows



In [83]:
trial_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [84]:
trial_df.columns

['ageofparticipant',
 'clinician',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result']

In [85]:
trial_df.dtypes

[('ageofparticipant', 'bigint'),
 ('clinician', 'struct<branch:string,name:string,role:string>'),
 ('drug_used', 'string'),
 ('experimentenddate', 'string'),
 ('experimentstartdate', 'string'),
 ('noofhourspassedatfirstreaction', 'bigint'),
 ('result', 'struct<conclusion:string,sideeffectsonparticipant:string>')]

## splitting of the nested json file

In [86]:
columns=['ageofparticipant',
 'clinician.branch',
 'clinician.name',
 'clinician.role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result.conclusion',
 'result.sideeffectsonparticipant']

In [87]:
trial_df.select(columns).show(5)

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

In [88]:
from pyspark.sql import functions as fn

In [94]:
trial_df.select([count(when(col(column).isNull(), column)).alias(column) for column in columns]).show()



+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|ageofparticipant|clinician.branch|clinician.name|clinician.role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|result.conclusion|result.sideeffectsonparticipant|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|               0|               0|             0|           109|        0|                0|                  0|                            73|               53|                              0|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+



In [62]:
#splitting the columns
new_trial_df=trial_df.select(columns)
new_trial_df.printSchema()

In [64]:
#renaming the columns
new_column_names={
'ageofparticipant':'age_of_participant',
'branch':'clinic_branch',
'name':'head_clinician',
'role':'assistants_role',
'experimentenddate':'experiment_end_date',
'experimentstartdate':'experiment_start_date',
'noofhourspassedatfirstreaction':'no_of_hours_passed_at_first_reaction',
'sideeffectsonparticipant':'side_effects_on_participant'
}

In [71]:
new_trial_df=new_trial_df.withColumnsRenamed(new_column_names)

In [72]:
new_trial_df.dtypes

[('age_of_participant', 'bigint'),
 ('clinic_branch', 'string'),
 ('head_clinician', 'string'),
 ('assistants_role', 'string'),
 ('drug_used', 'string'),
 ('experiment_end_date', 'string'),
 ('experiment_start_date', 'string'),
 ('no_of_hours_passed_at_first_reaction', 'bigint'),
 ('conclusion', 'string'),
 ('side_effects_on_participant', 'string')]

In [74]:
new_trial_df.describe().show()

+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------------+-------------+---------------------------+
|summary|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used| experiment_end_date|experiment_start_date|no_of_hours_passed_at_first_reaction|   conclusion|side_effects_on_participant|
+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------------+-------------+---------------------------+
|  count|              3586|         3586|          3586|           3477|     3586|                3586|                 3586|                                3513|         3533|                       3586|
|   mean|17.507250418293363|         null|          null|           null|     null|1.618381578137200...| 1.615813671834913...|                   44.89097637346997|         null

In [76]:
new_trial_df= new_trial_df.na.fill({'assistants_role': 'N/A', 'conclusion': 'N/A'})

In [77]:
new_trial_df.describe().show()

+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------------+-------------+---------------------------+
|summary|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used| experiment_end_date|experiment_start_date|no_of_hours_passed_at_first_reaction|   conclusion|side_effects_on_participant|
+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------------+-------------+---------------------------+
|  count|              3586|         3586|          3586|           3586|     3586|                3586|                 3586|                                3513|         3586|                       3586|
|   mean|17.507250418293363|         null|          null|           null|     null|1.618381578137200...| 1.615813671834913...|                   44.89097637346997|         null

## TRANSFORMATION

In [96]:
new_trial_df.printSchema()

root
 |-- age_of_participant: long (nullable = true)
 |-- clinic_branch: string (nullable = true)
 |-- head_clinician: string (nullable = true)
 |-- assistants_role: string (nullable = false)
 |-- drug_used: string (nullable = true)
 |-- experiment_end_date: string (nullable = true)
 |-- experiment_start_date: string (nullable = true)
 |-- no_of_hours_passed_at_first_reaction: long (nullable = true)
 |-- conclusion: string (nullable = false)
 |-- side_effects_on_participant: string (nullable = true)

